# 네이버 헤드라인 뉴스를 음성으로 알려주는 서비스

### 1) 음성 녹음
### 2) 음성을 텍스트로 변환 (STT 서비스 요청->텍스트 수신)
### 3) 텍스트에 '뉴스' 라는 키워드가 포함되어 있는지 확인
### 4) 네이버 헤드라인 뉴스 크롤링
### 5) 텍스트를 음성으로 변환(TTS 서비스 요청->음성 데이터 수신)
### 6) 음성 데이터 재생

## 0. 모듈 가져오기

In [86]:
import requests
import json
import speech_recognition as sr
import multiprocessing
from playsound import playsound
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver

## 1. 음성 녹음 함수

In [2]:
def RecordVoice():
    r = sr.Recognizer()
    with sr.Microphone(sample_rate=16000) as source:
        print("Speak Anything :")
        audio = r.listen(source)
        return audio

## 2. STT 서비스 처리 함수

In [3]:
def RequestSTT(audio):
    url = "https://kakaoi-newtone-openapi.kakao.com/v1/recognize"
    key = "58df5a2afd5c776e6d4e95f5cabf709e"
    headers = {
        "Content-Type": "application/octet-stream",
        "Authorization": "KakaoAK " + key,
    }
    res = requests.post(url, headers=headers, data=audio.get_raw_data())
    
    return res

## 3. 키워드 비교 함수

In [11]:
def CompareKeyword(data):
    json_string = data[data.index('{"type":"finalResult"'):data.rindex('}')+1]
    result = json.loads(json_string)['value']
    result = result.replace(" ","")
    if "몇시" in result:
        return 'CURRENT_TIME'
    elif "뉴스" in result:
        return 'NEWS'
    else:
        return 'UNKNOWN'

## 4-1. 현재 시각 구하는 함수

In [5]:
def ProcessCurrentTime(path):
    print('ProcessCurrentTime', path)
    now = datetime.today().strftime('%H:%M')
    data = f'<speak>현재 시각은<break time="150ms"/> \
        <say-as interpret-as="time" format="hm24"> {now} </say-as> 입니다</speak>'.encode('utf-8').decode('latin1')
    res = RequestTTS(path, data)
    if (res.status_code != 200):
        print(f'TTS 요청이 실패하였습니다.(code={res.status_code})')
        return -1
    else:
        return 0
    

## 4-2. 헤드라인 뉴스를 가져오는 함수

In [81]:
def ProcessNews(path):
    print('ProcessNews', path)
    newsList = []
    url = 'https://news.naver.com'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'}
    res = requests.get(url, headers=headers)
    if (res.status_code != 200):
        print(f'Request 요청이 실패하였습니다.(code={res.status_code})')
        return -1
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    data = soup.find('ul', class_='hdline_article_list').findAll('div', class_='hdline_article_tit')
    #newsList = data.findAll('a')
    text = '<speak>오늘의 뉴스를 알려드리겠습니다.'
    for n, i in enumerate(data):
        news = i.text.replace('\n', '').replace('  ', '')
        text += f'{n+1}번: {news}<break time="500ms"/>'
    text += '</speak>'
    text = text.encode('utf-8').decode('latin1')
    res = RequestTTS(path, text)
    if (res.status_code != 200):
        print(f'TTS 요청이 실패하였습니다.(code={res.status_code})')
        return -1
    else:
        return 0
    
    

## 5. TTS 서비스 처리 함수

In [82]:
def RequestTTS(path, data):
    print('RequestTTS', path)
    url = "https://kakaoi-newtone-openapi.kakao.com/v1/synthesize"
    key = "58df5a2afd5c776e6d4e95f5cabf709e"
    headers = {
        "Content-Type": "application/xml",
        "Authorization": "KakaoAK " + key,
    }
    res = requests.post(url, headers=headers, data=data)
    if (res.status_code != 200):
            print(f'TTS 요청이 실패하였습니다.(code={res.status_code})')
            return res
    else:
        with open(path, "wb") as f:
            f.write(res.content)
        return res

## 6. 음성 데이터 재생

In [83]:
def PlayAudio(path):
    print('PlayAudio', path)
    p = multiprocessing.Process(target=playsound, args=(path,))
    p.start()
    return p

## MAIN

In [84]:
if __name__ == "__main__":
    path = 'temp.mp3'
    audio = RecordVoice()
    res = RequestSTT(audio)
    keyword = CompareKeyword(res.text)
    if (keyword == 'UNKNOWN'):
        print('알수 없는 키워드입니다.')
        exit(1)
    elif (keyword == 'CURRENT_TIME'):
        res = ProcessCurrentTime(path)
        if (res == 0):
            p = PlayAudio(path)      
    elif (keyword == 'NEWS'):
        res = ProcessNews(path)
        if (res == 0):
            p = PlayAudio(path)      
    

Speak Anything :
ProcessNews temp.mp3
RequestTTS temp.mp3
PlayAudio temp.mp3
